# 03. Learning LangGraph - Agent Supervisor

In [2]:
!pip install --quiet -U langchain langchain_openai langgraph langchainhub langchain_experimental

You should consider upgrading via the 'c:\users\hrint\documents\ai-poc's\ai\scripts\python.exe -m pip install --upgrade pip' command.


In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-1106-preview")

In [5]:
from langchain_core.tools import tool


# dummy output
spglobal_desc = """The annual return of the S&P 500 index can vary significantly from year to year, reflecting the performance of the 500 largest companies listed on stock exchanges in the United States. Historically, the average annual return of the S&P 500, including dividends, has been around 7-10% after adjusting for inflation. However, this average includes periods of significant growth as well as periods of decline.\n\nTo give you a specific figure for a particular year, I would need the year in question. Since my knowledge is up to date only until early 2023, I can provide information up until the end of 2022. For example, the annual return for the S&P 500 in 2021 was approximately 26.89%, which was an exceptional year for the index.\n\nFor the most current annual return figures, you would need to look up the latest financial data or consult financial news sources that provide updates on the S&P 500's performance. Keep in mind that past performance is not indicative of future results, and investing in the stock market always carries risk."""


@tool("simple_retriever")
def simple_retrieval_tool(input: str) -> str:
    """Returns the answer based on a single document"""
    return spglobal_desc


@tool("multi_doc_retriever" )
def multi_doc_retrieval_tool(input: str) -> str:
    """Returns the answer based on information from multiple documents"""
    return spglobal_desc


tools = [simple_retrieval_tool, multi_doc_retrieval_tool]

In [6]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI

def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    # Each worker node will be given a name and some tools.
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

In [7]:
# agent node
def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

In [15]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

members = [
    'simple_retriever', 'multi_doc_retriever']
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  {members}. Given the following user request,"
    "for one question you should only one and return the output directly"
    "do not call the same tool again."
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)
# Our team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed
options = ["FINISH"] + members
# Using openai function calling can make output parsing easier for us
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))


supervisor_chain = (
    prompt
    | llm.bind_functions(functions=[function_def], function_call="route")
    | JsonOutputFunctionsParser()
)

In [22]:
import operator
from typing import Annotated, Sequence, TypedDict
import functools
from langgraph.graph import StateGraph, END


# The agent state is the input to each node in the graph
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always
    # be added to the current states
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The 'next' field indicates where to route to next
    next: str


simple_retriever_agent = create_agent(
    llm, [simple_retrieval_tool], "You are a single-source analyst. You provide answers based on a single document.")
simple_retriever_node = functools.partial(
    agent_node, agent=simple_retriever_agent, name="simple_retriever")

multi_doc_retriever_agent = create_agent(
    llm, [multi_doc_retrieval_tool], "You are a multi-source analyst. You provide answers based  on information from multiple documents.")
multi_doc_retriever_node = functools.partial(
    
    agent_node, agent=multi_doc_retriever_agent, name="multi_doc_retriever")

workflow = StateGraph(AgentState)
workflow.add_node("simple_retriever", simple_retriever_node)
workflow.add_node("multi_doc_retriever", multi_doc_retriever_node)
workflow.add_node("supervisor", supervisor_chain)

In [23]:
for member in members:
    # We want our workers to ALWAYS "report back" to the supervisor when done
    workflow.add_edge(member, "supervisor") # add one edge for each of the agents

# The supervisor populates the "next" field in the graph state
# which routes to a node or finishes
conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
# Finally, add entrypoint
workflow.set_entry_point("supervisor")

graph = workflow.compile()

In [24]:
final_response = graph.invoke(
    {
        "messages": [
            HumanMessage(
                content="what is a S&P 500 index retruns?")
        ]
    }
)
final_response

{'messages': [HumanMessage(content='what is a S&P 500 index retruns?'),
  HumanMessage(content='The S&P 500 index returns can vary significantly from year to year, reflecting the performance of the 500 largest companies listed on stock exchanges in the United States. Historically, the average annual return of the S&P 500, including dividends, has been around 7-10% after adjusting for inflation. This average includes periods of significant growth as well as periods of decline.\n\nFor example, the annual return for the S&P 500 in 2021 was approximately 26.89%. However, please note that to get the most current annual return figures, you would need to consult the latest financial data or financial news sources. Past performance is not indicative of future results, and investing in the stock market always carries risk.', name='simple_retriever')],
 'next': 'FINISH'}

In [21]:
final_response = graph.invoke(
    {
        "messages": [
            HumanMessage(
                content="what is a S&P 500 index retruns and compare it with factor roator index?")
        ]
    }
)
final_response

{'messages': [HumanMessage(content='what is a S&P 500 index retruns and compare it with factor roator index?'),
  HumanMessage(content='I apologize, but there seems to have been a mistake in retrieving information specifically about the "Factor Rotator Index." Unfortunately, I am unable to provide the comparison between the S&P 500 index returns and the Factor Rotator Index without having information about the latter.\n\nThe Factor Rotator Index is not a standard financial index like the S&P 500, and may refer to a specific factor-based strategy or product offered by a financial institution. Factor investing involves targeting specific drivers of return, such as value, momentum, size, volatility, and quality. A Factor Rotator Index would typically aim to capitalize on the performance of these factors and rotate among them based on certain criteria.\n\nTo compare the S&P 500 index returns with the Factor Rotator Index, one would need to have performance data on the Factor Rotator Index,